# Retrieval Augmented Generation (RAG):
chat wwith local LLM, For Your PDFs

In [1]:
import langchain
from langchain.document_loaders.pdf import PyPDFDirectoryLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

from langchain_community.embeddings.bedrock import BedrockEmbeddings
from langchain_community.embeddings.ollama import OllamaEmbeddings

from langchain_community.vectorstores import Chroma
import warnings
warnings.filterwarnings("ignore")

from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List

In [2]:
DATA_PATH= "data"
CHROMA_PATH = "chroma"
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

### Load and Split the documents:

In [3]:
def load_docs():
    doc_loader = PyPDFDirectoryLoader(DATA_PATH)
    return doc_loader.load()

In [4]:
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

### Count the embedding:

In [5]:
def get_embedding_function():
    #if we use AWS
    # embeddings = BedrockEmbeddings(
    #     credentials_profile_name="default",
    #     region_name="us-east-1")
    
    #if we use Ollama 
    embeddings= OllamaEmbeddings(model="nomic-embed-text")
    
    return embeddings
    

### Create the database:

In [6]:
# def calculate_chunk_ids(chunks):

#     # This will create IDs like "data/monopoly.pdf:6:2"
#     # Page Source : Page Number : Chunk Index

#     last_page_id = None
#     current_chunk_index = 0

#     for chunk in chunks:
#         source = chunk.metadata.get("source")
#         page = chunk.metadata.get("page")
#         current_page_id = f"{source}:{page}"

#         # If the page ID is the same as the last one, increment the index.
#         if current_page_id == last_page_id:
#             current_chunk_index += 1
#         else:
#             current_chunk_index = 0

#         # Calculate the chunk ID.
#         chunk_id = f"{current_page_id}:{current_chunk_index}"
#         last_page_id = current_page_id

#         # Add it to the page meta-data.
#         chunk.metadata["id"] = chunk_id

#     return chunks

In [7]:
# def add_to_chroma(chunks: list[Document]):
#     db= Chroma(persist_directory=CHROMA_PATH,
#                embedding_function=get_embedding_function()
#                )
#     # Calculate Page IDs.
#     chunks_with_ids = calculate_chunk_ids(chunks)

#     # Add or Update the documents.
#     existing_items = db.get(include=[])  # IDs are always included by default
#     existing_ids = set(existing_items["ids"])
#     print(f"Number of existing documents in DB: {len(existing_ids)}")

#     # Only add documents that don't exist in the DB.
#     new_chunks = []
#     for chunk in chunks_with_ids:
#         if chunk.metadata["id"] not in existing_ids:
#             new_chunks.append(chunk)

#     if len(new_chunks):
#         print(f"Adding new documents: {len(new_chunks)}")
#         new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
#         db.add_documents(new_chunks, ids=new_chunk_ids)
#         db.persist()
#     else:
#         print("No new documents to add!")

In [8]:
def calculate_chunk_ids_for_sublist(sublist):
    last_page_id = None
    current_chunk_index = 0

    for chunk in sublist:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        chunk.metadata["id"] = chunk_id

    return sublist

def parallelize_calculate_chunk_ids(chunks, num_workers=4):
    chunk_size = len(chunks) // num_workers
    chunk_sublists = [chunks[i:i + chunk_size] for i in range(0, len(chunks), chunk_size)]

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = list(executor.map(calculate_chunk_ids_for_sublist, chunk_sublists))

    # Flatten the list of results into a single list
    return [chunk for sublist in results for chunk in sublist]

# Example usage
# chunks = ...  # Your chunks data
# updated_chunks = parallelize_calculate_chunk_ids(chunks)

In [9]:
def add_to_chroma_parallel(chunks: List[Document], num_workers=4):
    db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=get_embedding_function()
    )

    # Calculate Page IDs.
    chunks_with_ids = parallelize_calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Function to check and collect new chunks
    def filter_new_chunks(chunk):
        if chunk.metadata["id"] not in existing_ids:
            return chunk
        return None

    # Parallelize filtering of new chunks
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        new_chunks = list(executor.map(filter_new_chunks, chunks_with_ids))

    # Filter out None values from the list
    new_chunks = [chunk for chunk in new_chunks if chunk is not None]

    if len(new_chunks):
        print(f"Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("No new documents to add!")

# Example usage
# add_to_chroma_parallel(chunks)

In [11]:
documents = load_docs()
chunks = split_documents(documents)
updated_chunks = parallelize_calculate_chunk_ids(chunks)
add_to_chroma_parallel(updated_chunks)

# def parallelize_load_and_process():
#     # Parallelize loading documents
#     with ThreadPoolExecutor() as executor:
#         future_docs = executor.submit(load_docs)
#         documents = future_docs.result()

#     # Parallelize splitting documents into chunks
#     with ThreadPoolExecutor() as executor:
#         future_chunks = executor.submit(split_documents, documents)
#         chunks = future_chunks.result()

#     # Parallelize adding chunks to Chroma
#     with ThreadPoolExecutor() as executor:
#         future_add_to_chroma = executor.submit(add_to_chroma, chunks)
#         future_add_to_chroma.result()

# parallelize_load_and_process()

Number of existing documents in DB: 85
Adding new documents: 2329


### Create qurey data:

In [ ]:
def query_rag(query_text: str):
    # Prepare the DB.
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)

    model = Ollama(model="llama3")
    response_text = model.invoke(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text

In [ ]:
if __name__ == "__main__":
    query= input("Please enter the query text: ")
    query_rag(query)

Response: According to the text, Cloud Computing is defined as an area that has its own characteristics which make it distinct from other research areas. The key characteristics mentioned are:

* Conceptional service-oriented characteristic
* Loose coupling and strong fault tolerance
* Ease of use user experience characteristic
* TCP/IP especially Internet based
* Virtualization
* High security

These characteristics help to distinguish Cloud Computing from other research areas such as HPC, virtualization, utility computing, and grid computing.
Sources: ['data\\TheCharacteristicsofCloudComputing_20140722_154207.pdf:4:1', 'data\\TheCharacteristicsofCloudComputing_20140722_154207.pdf:0:5', 'data\\TheCharacteristicsofCloudComputing_20140722_154207.pdf:0:0', 'data\\TheCharacteristicsofCloudComputing_20140722_154207.pdf:0:6', 'data\\TheCharacteristicsofCloudComputing_20140722_154207.pdf:4:0']
